In [15]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd

import os


In [2]:
JOHANNES_CLIENT_ID = 'ef739fe8683f4dada353b4260519e694'
JOHANNES_CLIENT_SECRET = '5076481d53f24b1384fc79754f35a82c'

CLIENT_ID = JOHANNES_CLIENT_ID
CLIENT_SECRET = JOHANNES_CLIENT_SECRET

```
!!! DISCLAIMER !!!
You must log out of spotify in your browser to get data from another user.
```

In [19]:
# some utils
def keep_keys(dictionary, keys):
    return {key: dictionary[key] for key in keys if key in dictionary}


def get_track_info(row):
    track_ID = row['song_ID']
    res = sp.audio_features(track_ID)
    keys = [
        'danceability',
        'energy',
        'loudness',
        'speechiness',
        'acousticness',
        'instrumentalness',
        'liveness',
        'valence',
        'tempo',
    ]
    fetures = keep_keys(res[0], keys=keys)
    # print(json.dumps(fetures, indent=2))
    return fetures

In [14]:
# fetch songs !!! Watch out for corret user -> check header !!!
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                                       client_secret=CLIENT_SECRET,
                                                       redirect_uri="http://localhost:3000",
                                                       scope="user-read-recently-played"))
results = sp.current_user_recently_played(limit=50)

for ind, title in enumerate(results['items'][0:5]):
    print(ind, title['played_at'], title['track']['name'])

0 2023-06-15T14:31:07.599Z Fire Meet Gasoline
1 2023-06-15T14:27:05.794Z The Greatest (feat. Kendrick Lamar)
2 2023-06-15T14:23:35.726Z Cheap Thrills (feat. Sean Paul)
3 2023-06-15T14:19:50.643Z No New Friends (feat. Sia, Diplo, and Labrinth)
4 2023-06-15T14:16:54.349Z Chandelier


In [20]:
fetched_songs = []
for item in results['items']:
    item_data = {
        "TIME_STAMP": item['played_at'],
        "song_ID": item['track']['id'],
        "song_Name": item['track']['name'],
        # TODO for all artists
        "artist": item['track']['artists'][0]['name'],
        "album": item['track']['album']['id'],
        "album_name": item['track']['album']['name'],
        "popularity": item['track']['popularity'],
        "explicit": item['track']['explicit'],
    }
    fetched_songs.append(item_data)
df = pd.DataFrame(fetched_songs)

sample_df = df.copy()
sample_df[[
    'danceability',
    'energy',
    'loudness',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
]] = sample_df.apply(get_track_info, axis=1, result_type='expand')


In [22]:
display(sample_df.head())

,TIME_STAMP,song_ID,song_Name,artist,album,album_name,popularity,explicit,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2023-06-15T14:31:07.599Z,2akkzRZ5HADuMZ03TlyxSR,Fire Meet Gasoline,Sia,6FdNvoO5sF4EKwCX9je1MH,1000 Forms Of Fear (Deluxe Version),51,False,0.418,0.764,-3.307,0.0559,0.00598,0.000000,0.1490,0.330,175.669
1,2023-06-15T14:27:05.794Z,7xHWNBFm6ObGEQPaUxHuKO,The Greatest (feat. Kendrick Lamar),Sia,2eV6DIPDnGl1idcjww6xyX,This Is Acting (Deluxe Version),71,False,0.668,0.725,-6.127,0.2660,0.01020,0.000479,0.0561,0.729,191.944
2,2023-06-15T14:23:35.726Z,2ONIJ40u0Attbm3miIIpNO,Cheap Thrills (feat. Sean Paul),Sia,2eV6DIPDnGl1idcjww6xyX,This Is Acting (Deluxe Version),67,False,0.592,0.788,-4.924,0.2010,0.05990,0.000003,0.1250,0.755,89.949
3,2023-06-15T14:19:50.643Z,6Dma0t0hOe6Bd6u5YRKF3n,"No New Friends (feat. Sia, Diplo, and Labrinth)",Sia,0ujHQ5WCLuKJQXOqXpGtpf,"LABRINTH, SIA & DIPLO PRESENT... LSD",62,False,0.725,0.703,-4.641,0.0466,0.06320,0.000251,0.1400,0.638,106.032
4,2023-06-15T14:16:54.349Z,4VrWlk8IQxevMvERoX08iC,Chandelier,Sia,3xFSl9lIRaYXIYkIn3OIl9,1000 Forms Of Fear,82,False,0.399,0.787,-2.880,0.0499,0.01970,0.000061,0.0685,0.572,117.089


In [26]:
# Save Dataframe
filePath = "."
fileName = "johannes_songs.csv"
sample_df.to_csv(os.path.join(filePath, fileName))